In [1]:
import os
import glob
import csv
import json
import pandas as pd

# Part 1: Prepare 2017-2018 Data
Raw Data Source: https://www.kaggle.com/datasnaek/youtube-new

## 1.1 Read Files

In [2]:
# Change directory to folder containing csv raw data
raw_data_path = os.path.join("Raw Data", "2017-2018")
os.chdir(raw_data_path)

# Print the current directory 
os.getcwd()

'E:\\Sem 9\\IST3134 Big Data Analytics in the Cloud\\Assignment (MapReduce)\\Raw Data\\2017-2018'

In [3]:
# Get all filenames with csv extension
extension = 'csv'
data_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [5]:
df_list = []

for file in data_filenames:
    temp_df = pd.read_csv(file, error_bad_lines=False)
    temp_df.drop(columns=['video_error_or_removed'], inplace=True)
    df_list.append(temp_df)

b'Skipping line 8155: expected 16 fields, saw 31\nSkipping line 8243: expected 16 fields, saw 31\nSkipping line 13554: expected 16 fields, saw 31\nSkipping line 14615: expected 16 fields, saw 31\nSkipping line 14855: expected 16 fields, saw 31\nSkipping line 15134: expected 16 fields, saw 31\n'
b'Skipping line 650: expected 16 fields, saw 30\nSkipping line 869: expected 16 fields, saw 30\nSkipping line 4594: expected 16 fields, saw 31\nSkipping line 4772: expected 16 fields, saw 25\nSkipping line 4817: expected 16 fields, saw 31\nSkipping line 5044: expected 16 fields, saw 31\nSkipping line 5414: expected 16 fields, saw 25\nSkipping line 5645: expected 16 fields, saw 25\nSkipping line 5853: expected 16 fields, saw 30\nSkipping line 8404: expected 16 fields, saw 25\nSkipping line 8442: expected 16 fields, saw 25\nSkipping line 8643: expected 16 fields, saw 31\nSkipping line 8672: expected 16 fields, saw 25\nSkipping line 9750: expected 16 fields, saw 25\nSkipping line 10006: expected 16

In [6]:
country_list = ["Canada", "Germany", "France", "Great Britain", "India", "Japan", "Korea", "Mexico", "Russia", "United States"]
index = 0

for df in df_list:
    # Add column "country"
    df["country"] = country_list[index]
    index += 1
    
    # Change date format for "trending_date"
    # Apply dummy time since time info not available
    df["trending_date"] = df['trending_date'].apply(lambda x: "20" + x[:2] + "-" + x[-2:] + "-" + x[3:5] + "T00:00:00.000Z")

## 1.2 Match Category

In [7]:
# Get all filenames with json extension
extension = 'json'
category_filenames = [i for i in glob.glob('*.{}'.format(extension))]
print(category_filenames)

['CA_category_id.json', 'DE_category_id.json', 'FR_category_id.json', 'GB_category_id.json', 'IN_category_id.json', 'JP_category_id.json', 'KR_category_id.json', 'MX_category_id.json', 'RU_category_id.json', 'US_category_id.json']


In [8]:
json_list = []

for file in category_filenames:
    
    with open(file) as f:
        temp_data = json.load(f)
        json_list.append(temp_data)

In [10]:
def extract_category(json_data):
    category_dict = {"category_id": [], "category": []}

    # Add category for id 29 (not available in json file)
    category_dict["category_id"].append(29)
    category_dict["category"].append("Nonprofits & Activism")
    
    # Access the list stored in "items"
    category_list = json_data["items"]

    # Loop through dict in category_list
    for i in range(len(category_list)):

        # Extract id
        category_id = int(category_list[i]["id"])
        category_dict["category_id"].append(category_id)

        # Extract category in nested dict
        category = category_list[i]["snippet"]["title"]
        category_dict["category"].append(category)
        
    # Convert dict to DataFrame
    category_df = pd.DataFrame.from_dict(category_dict)
    
    return category_df

In [11]:
category_df_list = []

for json_data in json_list:
    temp_df = extract_category(json_data)
    category_df_list.append(temp_df)

In [12]:
merged_list = []

for i in range(len(df_list)):
    left_join = pd.merge(df_list[i], category_df_list[i], on ='category_id', how ='left') 
    merged_list.append(left_join)

## 1.3 Rearrange Columns

In [40]:
# Create list to store new column position
column_names = ['video_id',
                'trending_date',
                'title',
                'channel_title',
                'category_id',
                'category',
                'country',
                'publish_time',
                'tags',
                'views',
                'likes',
                'dislikes',
                'comment_count',
                'thumbnail_link',
                'comments_disabled',
                'ratings_disabled',
                'description']

In [41]:
# Rearrange column position
for i in range(len(merged_list)):
    merged_list[i] = merged_list[i].reindex(columns=column_names)

## 1.4 Combine Files

In [42]:
# Combine all files in the list
combined_csv_1 = pd.concat(merged_list)

# Part 2: Prepare 2020-2021 Data 
Raw Data Source: https://www.kaggle.com/rsrishav/youtube-trending-video-dataset

## 2.1 Read Files

In [43]:
# Change directory to folder containing csv raw data
try:
    raw_data_path = os.path.join("Raw Data", "2020-2021")
    os.chdir(raw_data_path)
    
except FileNotFoundError:
    path_parent = os.path.dirname(os.getcwd())
    os.chdir(path_parent)
    raw_data_path = os.path.join("2020-2021")
    os.chdir(raw_data_path)

# Print the current directory 
os.getcwd()

'E:\\Sem 9\\IST3134 Big Data Analytics in the Cloud\\Assignment (MapReduce)\\Raw Data\\2020-2021'

In [44]:
# Get all filenames with csv extension
extension = 'csv'
data_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [21]:
print(data_filenames)

['CA_youtube_trending_data.csv', 'DE_youtube_trending_data.csv', 'FR_youtube_trending_data.csv', 'GB_youtube_trending_data.csv', 'IN_youtube_trending_data.csv', 'JP_youtube_trending_data.csv', 'KR_youtube_trending_data.csv', 'MX_youtube_trending_data.csv', 'RU_youtube_trending_data.csv', 'US_youtube_trending_data.csv']


In [45]:
df_list = []

for file in data_filenames:
    temp_df = pd.read_csv(file) 
    
    # Remove column "channelID"
    temp_df.drop(columns=["channelId"], inplace=True)
    
    # Rename columns to match 2017-2018 dataset
    temp_df.rename(columns={
        "channelTitle": "channel_title",
        "categoryId":  "category_id",
        "view_count": "views",
        "publishedAt":  "publish_time",
        }, inplace=True)
    
    df_list.append(temp_df)

In [46]:
country_list = ["Canada", "Germany", "France", "Great Britain", "India", "Japan", "Korea", "Mexico", "Russia", "United States"]
index = 0

# Add column "country"
for df in df_list:
    df["country"] = country_list[index]
    index += 1

## 2.2 Match Category

In [47]:
# Get all filenames with json extension
extension = 'json'
category_filenames = [i for i in glob.glob('*.{}'.format(extension))]
print(category_filenames)

['CA_category_id.json', 'DE_category_id.json', 'FR_category_id.json', 'GB_category_id.json', 'IN_category_id.json', 'JP_category_id.json', 'KR_category_id.json', 'MX_category_id.json', 'RU_category_id.json', 'US_category_id.json']


In [48]:
json_list = []

for file in category_filenames:
    
    with open(file) as f:
        temp_data = json.load(f)
        json_list.append(temp_data)

In [49]:
category_df_list = []

for json_data in json_list:
    temp_df = extract_category(json_data)
    category_df_list.append(temp_df)

In [50]:
merged_list = []

for i in range(len(df_list)):
    left_join = pd.merge(df_list[i], category_df_list[i], on ='category_id', how ='left')
    #left_join["category"].fillna("Missing", inplace = True) 
    merged_list.append(left_join)

## 2.3 Rearrange Columns

In [62]:
# Create list to store new column position
column_names = ['video_id',
                'trending_date',
                'title',
                'channel_title',
                'category_id',
                'category',
                'country',
                'publish_time',
                'tags',
                'views',
                'likes',
                'dislikes',
                'comment_count',
                'thumbnail_link',
                'comments_disabled',
                'ratings_disabled',
                'description']

In [63]:
# Rearrange column position
for i in range(len(merged_list)):
    merged_list[i] = merged_list[i].reindex(columns=column_names)

## 2.4 Combine Files

In [64]:
# Combine all files in the list
combined_csv_2 = pd.concat(merged_list)

# Part 3: Combine All Data

In [65]:
# Combine 2017-2018 & 2020-2021 data
combined_all = pd.concat([combined_csv_1, combined_csv_2])

# Go up one directory
path_parent = os.path.dirname(os.getcwd())
os.chdir(path_parent)

# Export to csv
combined_all.to_csv( "combined_all.csv", index=False, encoding='utf-8-sig')